# 汇聚层

实现汇聚层的前向传播

In [1]:
import torch
from torch import nn
from d2l import torch as d2l
'''
X: 输入的二维张量（矩阵），形状为 (高度, 宽度)
pool_size: 池化窗口的大小，格式为 (池化窗口高度, 池化窗口宽度)
mode: 池化模式，默认为 'max'（最大池化），可选 'avg'（平均池化）
'''
def pool2d(X, pool_size, mode='max'):
    # 解包池化窗口的高度和宽度
    p_h, p_w = pool_size
    # 创建输出张量Y，形状根据输入X和池化窗口计算得出
    # 假设：无填充(padding=0)，步长为1(stride=1)
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            # 最大池化：取池化窗口覆盖区域的最大值
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            # 平均池化：取池化窗口覆盖区域的平均值
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

验证二维最大汇聚层的输出<br>，![](./img/汇聚层.png)

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

验证平均汇聚层

In [3]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

填充和步幅

In [4]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4)) # (批量大小, 通道数, 高度, 宽度)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

深度学习框架中的步幅与汇聚窗口的大小相同

In [5]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

填充和步幅可以手动设定

In [6]:
'''
[[ 0,  0,  0,  0,  0,  0],
 [ 0,  0,  1,  2,  3,  0],
 [ 0,  4,  5,  6,  7,  0],
 [ 0,  8,  9, 10, 11,  0],
 [ 0, 12, 13, 14, 15,  0],
 [ 0,  0,  0,  0,  0,  0]]
 上下填充0，输出尺寸：2x2
'''
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

设定一个任意大小的矩形汇聚窗口，并分别设定填充和步幅的高度和宽度

In [7]:
pool2d = nn.MaxPool2d((2, 3), # 窗口：2行高，3列宽
                      stride=(2, 3), # 步长：垂直跳2行，水平跳3列 
                      padding=(0, 1)) # 填充：高度不填，宽度左右各填1列0
'''
[[ 0,  0,  1,  2,  3,  0],  ← 行0
 [ 0,  4,  5,  6,  7,  0],  ← 行1
 [ 0,  8,  9, 10, 11,  0],  ← 行2
 [ 0, 12, 13, 14, 15,  0]]  ← 行3
'''
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

汇聚层在每个输入通道上单独运算

In [8]:
'''
输入X（形状：1×1×4×4）
tensor([[[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11],
          [12, 13, 14, 15]]]])
tensor([[[[ 1,  2,  3,  4],
          [ 5,  6,  7,  8],
          [ 9, 10, 11, 12],
          [13, 14, 15, 16]]]])
'''
# 沿通道维度（dim=1）拼接
# torch.cat是张量结构的扩展，zip是元素级别的重组
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [9]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])

结合之前的例子验证以之前的池化参数`nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))` 为例：<br>
输入 X 的高度 H=4，宽度 W=4；窗口尺寸 $k_{H}=2$，$k_{W}=3$；步长 $s_{H}=2$，$s_{W}=3$；填充 $p_{H}=0$，$p_{W}=1$。计算输出高度 $O_{H}$：$O_{H} = \left\lfloor \frac{4 + 2×0 - 2}{2} \right\rfloor + 1 = \left\lfloor \frac{2}{2} \right\rfloor + 1 = 1 + 1 = 2$<br>
计算输出宽度 $O_{W}$：$O_{W} = \left\lfloor \frac{4 + 2×1 - 3}{3} \right\rfloor + 1 = \left\lfloor \frac{3}{3} \right\rfloor + 1 = 1 + 1 = 2$